In [26]:
pwd

'C:\\Users\\HP\\python_pintech\\02 DataType and Date Load'

In [12]:
import requests #Used to service API connection
from lxml import html #Used to parse XML
from bs4 import BeautifulSoup #Used to read XML table on webpage
import pandas as pd
import numpy as np
import wget
from common import cFunction as cf


# get dataList from filesystem to load and write
#dataList = pd.read_excel("../../data/inbound/dataList.xlsx")

# get dataList from spreadsheet to load and write
dataList = pd.read_csv("https://docs.google.com/spreadsheets/d/1VngqG-m7G8k1587c21MZoheR1Fz3amp1mJtiBvA1Jb0/export?format=csv&gid=0")
print("### The total number of target data is " + str(len(dataList)))

### The total number of target data is 417


In [13]:
# Filtering -> get dataList only defined url
dataList = dataList[dataList["제공항목(데이터셋)"] == '한국감정원 아파트거래 현황']
print(dataList[dataList["제공항목(데이터셋)"] == '한국감정원 아파트거래 현황'])
print("### The total number of filtered data is " + str(len(dataList)))

      번호   접속 사이트      제공항목(데이터셋)                 폴더명  작업자  시작일  종료일  검토자  \
319  320  공공데이터포털  한국감정원 아파트거래 현황  319_한국감정원 아파트거래 현황  NaN  NaN  NaN  NaN   

    진행상태   비고          ...            \
319  NaN  NaN          ...             

                                               파라미터 제공방식.1 데이터타입  \
319  startmonth=201402&endmonth=201407&region=48000    api   xml   

                                                  참고문서 참고문서타입  주기  \
319  https://www.data.go.kr/comm/file/download.do?a...   docx NaN   

               비고_파라미터설명 완료여부     API PGM ID                    파일명  
319  201402&201409&27000  NaN  TB_OD10319.py  TB_OD10319_yyyymm.csv  

[1 rows x 31 columns]
### The total number of filtered data is 1


In [14]:
outPath = "../../data/outbound/"
folderList = dataList["폴더명"].tolist()

In [15]:
for i in folderList:
    cf.createFolder(outPath+i)

In [16]:
dataList = dataList.fillna("")
dataList = dataList.reset_index(drop=True)
dataList

,번호,접속 사이트,제공항목(데이터셋),폴더명,작업자,시작일,종료일,검토자,진행상태,비고,...,파라미터,제공방식.1,데이터타입,참고문서,참고문서타입,주기,비고_파라미터설명,완료여부,API PGM ID,파일명
0,320,공공데이터포털,한국감정원 아파트거래 현황,319_한국감정원 아파트거래 현황,,,,,,,...,startmonth=201402&endmonth=201407&region=48000,api,xml,https://www.data.go.kr/comm/file/download.do?a...,docx,,201402&201409&27000,,TB_OD10319.py,TB_OD10319_yyyymm.csv


In [18]:
# get dataList to load and write
inputUrl = dataList.loc[dataCount, "사이트"]
inputKey = dataList.loc[dataCount, "서비스키"]
inputParameter = dataList.loc[dataCount, "파라미터"]
inputFolder = dataList.loc[dataCount, "폴더명"]
inputFile = dataList.loc[dataCount, "서비스명"]
inputDataType = dataList.loc[dataCount, "데이터타입"]
inputRefUrl = dataList.loc[dataCount, "참고문서"]
inputRefType = dataList.loc[dataCount, "참고문서타입"]
inputbParameter = dataList.loc[dataCount, "비고_파라미터설명"]
print(type(inputbParameter))

<class 'str'>


In [19]:
    url = cf.makeURL(inputUrl,inputKey,"")
    print("fullUrl is " + url)

fullUrl is http://openapi.kab.co.kr/OpenAPI_ToolInstallPackage/service/rest/AptTradingStateSvc/getAptTrdStateCaseOfNbr?serviceKey=Z5J2q693N6mpcWIY9wSZqKgJXsAcan78Wc8dorriZfblBHoCZSqawyAVOgvykleO26uY39699xPbDd4ENw6s1w%3D%3D


In [20]:
def operatorXmlProcess(url, parameter, bParameter):
    splitedbP = bParameter.split('&')
    lastUrl = url
    parameterList = parameter.split('&')
    for i in  range(0, len(parameterList)):
        lastUrl += '&' + (parameterList[i].split('=')[0] + '=' + splitedbP[i]) 
        
    print(lastUrl)
    response = requests.get(lastUrl)
    # Check if page is up
    if response.status_code == 200:
        # Convert webpage to %Data
        Data = BeautifulSoup(response.text, 'lxml-xml')
        result = []
        rows = 0
        columnName = []
        # search Item all item tag
        iterData = Data.find_all('item')
        for item in iterData:
            item_list = []
            # Fill the value in one row
            for tag in item.find_all():
                try:
                    tagname = tag.name
                    if rows == 0:
                        columnName.append(tagname)
                    item_list.append(item.find(tagname).text)
                except Exception as e:
                    print("This row will be ignored. ", item_list)
            rows = rows + 1
            result.append(item_list)
    finalResult = pd.DataFrame(result)
    finalResult.columns = columnName
    print(finalResult)
    return finalResult

In [21]:
newDF = pd.DataFrame()
if (inputDataType == "xml"):
    newDF = operatorXmlProcess(url, inputParameter, inputbParameter)
elif(inputDataType == "json"):
    newDF = cf.jsonProcess(url)
elif(inputDataType == "csv"):
    newDF = cf.csvProcess(url)

http://openapi.kab.co.kr/OpenAPI_ToolInstallPackage/service/rest/AptTradingStateSvc/getAptTrdStateCaseOfNbr?serviceKey=Z5J2q693N6mpcWIY9wSZqKgJXsAcan78Wc8dorriZfblBHoCZSqawyAVOgvykleO26uY39699xPbDd4ENw6s1w%3D%3D&startmonth=201402&endmonth=201409&region=27000
  regionCd regionNm                                              rsRow
0    27000       대구  201402,6432|201403,5984|201404,6371|201405,512...


In [22]:
fullOutPath = outPath+inputFolder+"/"+inputFolder+inputFile+".csv"
print(fullOutPath)

../../data/outbound/319_한국감정원 아파트거래 현황/319_한국감정원 아파트거래 현황.csv


In [23]:
try:
    newDF.to_csv(fullOutPath, index=False, encoding="ms949")
except Exception as x:
    print(x)

In [24]:
fullOutRefPath = outPath + inputFolder + "/" + inputFolder + inputFile + "."+inputRefType
try:
    wget.download(inputRefUrl, fullOutRefPath)
except Exception as e:
    print(inputFolder+"참고문서 Error")
    print(e)
    pass

100% [........................................................] 550930 / 550930

In [25]:
finalData = {'regionCd' : [], 'regionNm' : [], 'rsRow' : []}
finalResult = pd.DataFrame(finalData)
finalResult

,regionCd,regionNm,rsRow
